In [2]:
# install a package
install.packages("RPostgreSQL")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# load libraries
library(DBI)            # R Database Interface
library(RPostgreSQL)    # ~ the driver to PostgreSQL
library(dplyr)

In [6]:
con <- dbConnect(
    PostgreSQL(),                                   # the driver -> to let R set any configs for a PostgreSQL server
    host = "floppy.db.elephantsql.com",             # the server from ElephantSQL
    dbname = "pdvnycgh",                            # user & default database from Elephant SQL
    port = 5432,                                    # the default port of PostgreSQL -> 5432
    user = "pdvnycgh",                              # same as dbname
    password = "DPWZGfm-gmhETtsPxTWxAaQmky6haki_"
)

con

<PostgreSQLConnection>

In [34]:
# create mock-up tables
fav_series <- data.frame(
    series_id = 1:5,
    series = c("Dark", "Twenty Five Twenty One", "First Love", "Black Mirror", "The Chestnut Man"),
    score = c(10, 9, 8, 7, 8)
)

genres <- data.frame(
    genre_id = 1:7,
    genre = c("Drama", "Action", "Mystery", "Comedy", "Crime", "Romance", "Dystopia")
)

bridge <- data.frame(
    series_id = c(1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5),
    genre_id = c(1, 3, 1, 4, 6, 1, 6, 1, 7, 2, 5)
)

In [37]:
# write tables to the database server
dbWriteTable(con, "fav_series", fav_series, row.names=FALSE)    # row names are ignored
dbWriteTable(con, "genres", genres, row.names=FALSE)
dbWriteTable(con, "bridge", bridge, row.names=FALSE)

[1] TRUE

[1] TRUE

[1] TRUE

In [38]:
# list all tables from the database server
dbListTables(con)

[1] "fav_series" "genres"     "bridge"

In [45]:
# get data from the database server
my_series_info <- dbGetQuery(con, 
                "SELECT 
                    se.series,
                    se.score,
                    ge.genre
                FROM fav_series AS se 
                JOIN bridge AS br ON se.series_id = br.series_id
                JOIN genres AS ge ON br.genre_id = ge.genre_id
                ORDER BY 1")
my_series_info

,series,score,genre
,<chr>,<dbl>,<chr>
1,Black Mirror,7,Drama
2,Black Mirror,7,Dystopia
3,Dark,10,Mystery
4,Dark,10,Drama
5,First Love,8,Romance
6,First Love,8,Drama
7,The Chestnut Man,8,Crime
8,The Chestnut Man,8,Action
9,Twenty Five Twenty One,9,Romance


In [47]:
# get data from the database server
# use dplyr to manipulate data
my_series <- dbGetQuery(con, "SELECT * FROM fav_series")
my_series %>%
  mutate(language = c("German", "Korean", "Japanese", "English", "Danish"))

,series_id,series,score,language
,<int>,<chr>,<dbl>,<chr>
1,1,Dark,10,German
2,2,Twenty Five Twenty One,9,Korean
3,3,First Love,8,Japanese
4,4,Black Mirror,7,English
5,5,The Chestnut Man,8,Danish


In [48]:
# !!! CLOSE connection
dbDisconnect(con)

[1] TRUE